In [ ]:
#Library yang digunakan
!pip install sastrawi
!pip install beautifulsoup4
!pip install anyascii

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import yang diperlukan
import string
import re
import pandas as pd

#nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

#sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#beautyfulsoup4
from bs4 import BeautifulSoup

#anyascii
from anyascii import anyascii

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def tokenized_kalimat(berita):
  test_raw_data = re.sub(r"(\\n|\\r)", r"\n", berita)
  test_raw_data = re.sub(r"(\\t|\\f|\\v)", r" ", test_raw_data)

  # Ekstraksi konten dari HTML Rich Text dengan BeautifulSoup4
  soup = BeautifulSoup(test_raw_data, "html.parser")
  news_html_free = soup.get_text()

  # Ubah semua karakter unicode menjadi ASCII yang paling mendekati
  news_html_free = anyascii(news_html_free)

  #tokenizing kalimat
  sent_tokenized = nltk.tokenize.sent_tokenize(news_html_free)

  return sent_tokenized

Fungsi dibawah digunakan untuk mempreproses datanya

In [ ]:
def preprocess_data(kalimat):
  sentence_processed = []

  #mengubah kalimat menjadi huruf kecil
  lower_case = kalimat.lower()

  #menghilangkan kelebihan whitespace/karakter kosong
  whitespace_removed = lower_case.strip()

  # menghilangkan tanda baca
  punctuation_removed = whitespace_removed.translate(str.maketrans("","",string.punctuation))

  word_tokenized = nltk.tokenize.word_tokenize(punctuation_removed)

  return word_tokenized

Fungsi dibawah adalah formula jaccard yang digunakan untuk menghitung similarity setiap kata.

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

berikut adalah fungsi untuk melakukan perhitungan similaritas setiap kata terhadap data yang dimiliki 

In [ ]:
def hitung_similarity(hasil_preproses, data):
  y = 0
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  word_send = []

  for kalimat in hasil_preproses:

    words_temp = []
    words_berita = preprocess_data(kalimat)
    
    for words in words_berita:
      is_true = True
      y = y + 1
      if stemmer.stem(words)[0] in CONSTANT:
        for data_words in data:
          if abs(len(words) - len(data_words)) <= 1:
            if stemmer.stem(words) == stemmer.stem(data_words):
              x = jaccard_similarity([*data_words], [*words])
              if x >= 0.75 and x <= 0.99:
                is_true = False
              else: 
                pass
            else: 
              pass
      else:
        pass
      words_temp.append({'words': words, 'is_true':is_true})

    word_send.append(words_temp)
  return word_send

In [ ]:
#Contoh Berita
# test_raw_data = """<p>kalimat labil pertama</p>
# <p>kalimat kedua &lt &amp &#342;</p>
# <p>kalimat&nbsp;ketiga & ketiga juga</p>
# <p>kalimat &nbsp; keempat</p>
# <p>kalimat <b>kel<i>i</i>ma</b></p>
# <p>kalimat keenam mempasarkan</a>
# """
!pip install docx2txt
import docx2txt

# berita = docx2txt.process("beritatest1.docx")
berita = 'Budi berhasil mensamarkan bekas luka dengan mentarik'


#Konstanta diperlukan karena peluluhan hanya terjadi pada kata dengan kata dasar S,P,T, dan K.
CONSTANT = ['s','p','t','k']

#Membaca dataset yang telah dicrawl, merupakan kata-kata yang benar.
dataset = pd.read_excel('kata_yang_benar.xlsx')
data = dataset["kata benar"].to_numpy()
# data = ['mengatakan','penagihan']

# pakai looping zip(t1,t2) untuk return datanya

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sentence_tokenized = tokenized_kalimat(berita)
print(sentence_tokenized)

['Budi berhasil mensamarkan bekas luka dengan mentarik']


In [ ]:
x = hitung_similarity(sentence_tokenized, data)
print(x)

test similarity kata ke 3 :  0.75
Kata mensamarkan tidak sesuai dengan kata menyamarkan
test similarity kata ke 7 :  0.875
Kata mentarik tidak sesuai dengan kata menarik
[[{'words': 'budi', 'is_true': True}, {'words': 'berhasil', 'is_true': True}, {'words': 'mensamarkan', 'is_true': False}, {'words': 'bekas', 'is_true': True}, {'words': 'luka', 'is_true': True}, {'words': 'dengan', 'is_true': True}, {'words': 'mentarik', 'is_true': False}]]
